In [1]:
import os
os.environ['SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL'] = 'True'
import subprocess
subprocess.run(['pip', 'install', 'sklearn'])

CompletedProcess(args=['pip', 'install', 'sklearn'], returncode=0)

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Input, Dense, Flatten
import pandas as pd
import numpy as np
import fastparquet as fp
import time
from sklearn.metrics import r2_score, root_mean_squared_error
import matplotlib.pyplot as plt
import jax
import tensorflow as tf
keras.utils.set_random_seed(1)

In [3]:
from sigtkan_Tristan import SigTKAN
from tkan import TKAN

In [4]:
df = pd.read_parquet('data.parquet')
df = df[(df.index >= pd.Timestamp('2020-01-01')) & (df.index < pd.Timestamp('2023-01-01'))]
assets = ['BTC', 'ETH', 'ADA', 'XMR', 'EOS', 'MATIC', 'TRX', 'FTM', 'BNB', 'XLM', 'ENJ', 'CHZ', 'BUSD', 'ATOM', 'LINK', 'ETC', 'XRP', 'BCH', 'LTC']
df = df[[c for c in df.columns if 'quote asset volume' in c and any(asset in c for asset in assets)]]
df.columns = [c.replace(' quote asset volume', '') for c in df.columns]
df = df.head(1000)
display(df)

,BTC,ADA,XMR,EOS,CHZ,MATIC,TRX,ENJ,FTM,BNB,XLM,BUSD,ATOM,LTC,LINK,ETC,ETH,XRP,BCH
group,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,3.675857e+06,3.818918e+04,45395.983051,9.477858e+04,817.146319,31003.791035,4.819934e+05,15241.945783,1165.788613,8.498617e+05,9.460820e+03,1.352376e+04,3.198697e+04,1.165827e+05,2.428117e+04,5.648840e+04,1.000930e+06,2.579254e+05,1.782587e+05
2020-01-01 01:00:00,6.365953e+06,5.135701e+04,33483.951528,5.932921e+05,886.460339,84465.335718,5.336686e+05,11896.843688,413.844612,7.405759e+05,3.714191e+04,2.531605e+04,8.177767e+04,2.830715e+05,5.119098e+04,1.821021e+05,1.474278e+06,4.520609e+05,6.153210e+05
2020-01-01 02:00:00,4.736719e+06,3.616426e+04,15732.553860,2.667326e+05,1819.795050,113379.718506,3.870500e+05,30109.770521,3559.965968,1.039091e+06,1.687882e+04,1.390886e+04,1.957312e+05,2.402871e+05,2.872176e+04,1.340634e+05,9.940256e+05,4.414948e+05,2.215356e+05
2020-01-01 03:00:00,5.667367e+06,2.444995e+04,25751.054662,1.245166e+05,2979.655803,41771.707995,4.507721e+05,6732.833578,4076.415482,4.975018e+05,9.049223e+03,2.251969e+04,1.201133e+05,1.613043e+05,2.959622e+04,1.310942e+05,6.473610e+05,1.886061e+05,3.971860e+05
2020-01-01 04:00:00,3.379094e+06,4.450267e+04,62955.628253,4.218197e+05,1023.388675,22254.756114,2.847890e+05,846.938455,633.367505,4.751285e+05,7.254260e+03,1.122460e+04,1.998917e+04,2.214516e+05,5.451437e+04,1.349371e+05,4.430067e+05,2.279373e+05,3.164991e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-02-11 11:00:00,1.438900e+07,2.965103e+05,73609.057536,8.430289e+05,9905.938130,234890.291193,6.946113e+05,44091.548613,5253.565829,2.060869e+06,4.312171e+05,6.804777e+05,3.642363e+05,1.213942e+06,2.417881e+06,9.594110e+05,2.859655e+06,1.167976e+06,1.936994e+06
2020-02-11 12:00:00,3.338677e+07,8.930125e+05,197514.376161,1.616912e+06,26623.483445,358801.206634,9.011416e+05,50717.331631,87151.192491,4.607640e+06,4.146479e+05,9.643264e+05,1.531237e+06,2.139694e+06,3.088998e+06,1.936027e+06,5.396577e+06,2.372525e+06,3.687622e+06
2020-02-11 13:00:00,2.599866e+07,4.148004e+05,198932.907884,1.686487e+06,59221.774114,374679.172660,4.930206e+05,26798.314578,14804.652222,2.158669e+06,3.196139e+05,1.138288e+06,4.748964e+05,9.989755e+05,3.447916e+06,6.709845e+05,3.749217e+06,1.116360e+06,1.535185e+06


In [5]:
class MinMaxScaler:
    def __init__(self, feature_axis=None, minmax_range=(0, 1)):
        """
        Initialize the MinMaxScaler.
        Args:
        feature_axis (int, optional): The axis that represents the feature dimension if applicable.
                                      Use only for 3D data to specify which axis is the feature axis.
                                      Default is None, automatically managed based on data dimensions.
        """
        self.feature_axis = feature_axis
        self.min_ = None
        self.max_ = None
        self.scale_ = None
        self.minmax_range = minmax_range # Default range for scaling (min, max)

    def fit(self, X):
        """
        Fit the scaler to the data based on its dimensionality.
        Args:
        X (np.array): The data to fit the scaler on.
        """
        if X.ndim == 3 and self.feature_axis is not None:  # 3D data
            axis = tuple(i for i in range(X.ndim) if i != self.feature_axis)
            self.min_ = np.min(X, axis=axis)
            self.max_ = np.max(X, axis=axis)
        elif X.ndim == 2:  # 2D data
            self.min_ = np.min(X, axis=0)
            self.max_ = np.max(X, axis=0)
        elif X.ndim == 1:  # 1D data
            self.min_ = np.min(X)
            self.max_ = np.max(X)
        else:
            raise ValueError("Data must be 1D, 2D, or 3D.")

        self.scale_ = self.max_ - self.min_
        return self

    def transform(self, X):
        """
        Transform the data using the fitted scaler.
        Args:
        X (np.array): The data to transform.
        Returns:
        np.array: The scaled data.
        """
        X_scaled = (X - self.min_) / self.scale_
        X_scaled = X_scaled * (self.minmax_range[1] - self.minmax_range[0]) + self.minmax_range[0]
        return X_scaled

    def fit_transform(self, X):
        """
        Fit to data, then transform it.
        Args:
        X (np.array): The data to fit and transform.
        Returns:
        np.array: The scaled data.
        """
        return self.fit(X).transform(X)

    def inverse_transform(self, X_scaled):
        """
        Inverse transform the scaled data to original data.
        Args:
        X_scaled (np.array): The scaled data to inverse transform.
        Returns:
        np.array: The original data scale.
        """
        X = (X_scaled - self.minmax_range[0]) / (self.minmax_range[1] - self.minmax_range[0])
        X = X * self.scale_ + self.min_
        return X

def generate_data(df, sequence_length, n_ahead = 1):
    #Case without known inputs
    scaler_df = df.copy().shift(n_ahead).rolling(24 * 14).median()
    tmp_df = df.copy() / scaler_df
    tmp_df = tmp_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    scaler_df = scaler_df.iloc[24 * 14 + n_ahead:].fillna(0.)
    def prepare_sequences(df, scaler_df, n_history, n_future):
        X, y, y_scaler = [], [], []
        num_features = df.shape[1]
        
        # Iterate through the DataFrame to create sequences
        for i in range(n_history, len(df) - n_future + 1):
            # Extract the sequence of past observations
            X.append(df.iloc[i - n_history:i].values)
            # Extract the future values of the first column
            y.append(df.iloc[i:i + n_future,0:1].values)
            y_scaler.append(scaler_df.iloc[i:i + n_future,0:1].values)
        
        X, y, y_scaler = np.array(X), np.array(y), np.array(y_scaler)
        return X, y, y_scaler
    
    # Prepare sequences
    X, y, y_scaler = prepare_sequences(tmp_df, scaler_df, sequence_length, n_ahead)
    
    # Split the dataset into training and testing sets
    train_test_separation = int(len(X) * 0.8)
    X_train_unscaled, X_test_unscaled = X[:train_test_separation], X[train_test_separation:]
    y_train_unscaled, y_test_unscaled = y[:train_test_separation], y[train_test_separation:]
    y_scaler_train, y_scaler_test = y_scaler[:train_test_separation], y_scaler[train_test_separation:]
    
    # Generate the data
    X_scaler = MinMaxScaler(feature_axis=2)
    X_train = X_scaler.fit_transform(X_train_unscaled)
    X_test = X_scaler.transform(X_test_unscaled)
    
    y_scaler = MinMaxScaler(feature_axis=2)
    y_train = y_scaler.fit_transform(y_train_unscaled)
    y_test = y_scaler.transform(y_test_unscaled)
    
    y_train = y_train.reshape(y_train.shape[0], -1) 
    y_test = y_test.reshape(y_test.shape[0], -1)
    return X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test

In [6]:
early_stopping_callback = lambda : tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.00001,    
    patience=6,         
    mode="min",
    restore_best_weights=True,
    start_from_epoch=3, 
    verbose=1,
)

lr_callback = lambda : tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=3,
    mode="min",
    min_delta=0.0001,  
    min_lr=0.000025,
    verbose=1,
)

callbacks = lambda : [early_stopping_callback(), lr_callback(), tf.keras.callbacks.TerminateOnNaN()]

In [7]:
n_aheads = [1]
models = [
    "TKAN",
    "SigTKAN",
]

N_MAX_EPOCHS = 1000
BATCH_SIZE = 128

callbacks = lambda : [
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3, verbose=1),
    tf.keras.callbacks.TerminateOnNaN()
]

results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
results_rmse = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
time_results = {model: {n_ahead: [] for n_ahead in n_aheads} for model in models}
for n_ahead in n_aheads:
    sequence_length = max(45, 5 * n_ahead)
    X_scaler, X_train, X_test, X_train_unscaled, X_test_unscaled, y_scaler, y_train, y_test, y_train_unscaled, y_test_unscaled, y_scaler_train, y_scaler_test = generate_data(df, sequence_length, n_ahead)
    
    for model_id in models:
        
        for run in range(10):

            if model_id == 'TKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    TKAN(100, return_sequences=True),
                    TKAN(100, sub_kan_output_dim = 20, sub_kan_input_dim = 20, return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            elif model_id == 'SigTKAN':
                model = Sequential([
                    Input(shape=X_train.shape[1:]),
                    SigTKAN(100, 2, dropout=0., return_sequences=True),
                    SigTKAN(100, 2, dropout=0., return_sequences=False),
                    Dense(units=n_ahead, activation='linear')
                ], name = model_id)
            else:
                raise ValueError
            
            optimizer = keras.optimizers.Adam(0.001)
            model.compile(optimizer=optimizer, loss='mean_squared_error', jit_compile=True)
            if run==0:
                model.summary()
                
            # Fit the model
            start_time = time.time()
            history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=N_MAX_EPOCHS, validation_split=0.2, callbacks=callbacks(), shuffle=True, verbose = False)
            end_time = time.time()
            time_results[model_id][n_ahead].append(end_time - start_time)
            # Evaluate the model on the test set
            preds = model.predict(X_test, verbose=False)
            r2 = r2_score(y_true=y_test, y_pred=preds)
            print(end_time - start_time, r2)
            rmse = root_mean_squared_error(y_true=y_test, y_pred=preds)
            results[model_id][n_ahead].append(r2)
            results_rmse[model_id][n_ahead].append(rmse)
    
            del model
            del optimizer
                

print('R2 scores')
print('Means:')
display(pd.DataFrame({model_id: {n_ahead: np.mean(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.mean(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Std:')
display(pd.DataFrame({model_id: {n_ahead: np.std(results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(results_rmse[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in results_rmse.keys()}))
print('Training Times')
display(pd.DataFrame({model_id: {n_ahead: np.mean(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))
display(pd.DataFrame({model_id: {n_ahead: np.std(time_results[model_id][n_ahead]) for n_ahead in n_aheads} for model_id in time_results.keys()}))

Model: "TKAN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tkan (TKAN)                     │ (None, 45, 100)        │        41,534 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tkan_1 (TKAN)                   │ (None, 100)            │        67,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 109,535 (427.87 KB)

 Trainable params: 109,145 (426.35 KB)

 Non-trainable params: 390 (1.52 KB)

...

,TKAN,SigTKAN
1,0.055493,0.01819


,TKAN,SigTKAN
1,0.003244,0.001062


Training Times


,TKAN,SigTKAN
1,20.046711,61.695707


,TKAN,SigTKAN
1,3.764072,2.05045
